In [1]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO


In [39]:
pd_csv = pd.read_csv("output-37-pandas.csv")
for i in range(3):
    if(pd_csv.iloc[i].astype(str).str.contains("Maturity Date").any() == True):
        print("True")

True


In [40]:
for k in range(10):
    pd_csv = pd.read_csv("output-"+str(k)+"-pandas.csv")
    for i in range(3):
        if(pd_csv.iloc[i].astype(str).str.contains("Maturity Date").any() == True):
            print("True")

In [81]:
pd_csv = pd.read_csv("output-37-pandas.csv")
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
pd_csv.reset_index()
# Find out where portfolio company is
for col in pd_csv.columns:
    pd_csv[col] = pd_csv[col].astype(str).str.replace('  ', ' ')
    portfolio_index_list = pd_csv.index[pd_csv[col].astype(str).str.contains("Portfolio Company")].to_list()
    if (len(portfolio_index_list) > 0):
        break
portfolio_index = portfolio_index_list[0]
# Checking if all the column are in the row with portfolio and adding them if they are not.
if (pd_csv.iloc[portfolio_index].str.contains('Principal').any() == False):
    for j in range(pd_csv.shape[1]):
        if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
            pd_csv.iat[portfolio_index, j] = 'Principal'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
            pd_csv.iat[portfolio_index, j] = 'Cost'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
            pd_csv.iat[portfolio_index, j] = 'Fair Value'
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Principal').any() == True):
    pd_csv = pd_csv.iloc[portfolio_index:]

#Identifying levels in the dataframe
listOfLevels = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if(pd.notnull(pd_csv.iat[i, j])):
            #print(pd_csv.iat[i, j])
            if (('Investments' in str(pd_csv.iat[i, j])) and 
                ('Subtotal' not in pd_csv.iat[i, j]) and 
                ('Total' not in pd_csv.iat[i, j])):
                if(pd_csv.iloc[i].isnull().sum()==pd_csv.shape[1]-1):
                    #print(pd_csv.iloc[i])
                    listOfLevels.append(i)
print(listOfLevels)
                    
#Adding the Levels to an array
newValues = []
newValues.append("Levels")
for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if(i==len(listOfLevels)-1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    for x in range(beginIndex, endIndex):
        newValues.append(value)

#Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)
pd_csv = pd_csv.drop(listOfLevels)

#Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            #print(pd_csv.iloc[i,:])
            subtotal_list.append(i)
            
print("Subtotal",subtotal_list)
#Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list],inplace=True)

#Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
#Dropping everything after the total rows
try:
    pd_csv = pd_csv.iloc[:total_list[0]]
except:
    print("Total not found")
    
#Identifying index of Type of Investment
i = 0
for j in range(pd_csv.shape[1]):
    if ('Type of Investment' in str(pd_csv.iat[i, j])):
        type_of_investment_index = j
print("Type Investment",type_of_investment_index)

# Adding company for the middle values where there are null values
for i in range(2, pd_csv.shape[0]):
    for j in range(type_of_investment_index):
        #Value to check if row has alphabets or not
        row_alphabet_check = False
        #Checking if the current cell is null or nan
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j]=='nan')):
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                #If cell above is also null let it be
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    #Check if there are alphabets in the cell above current cell
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            #If there is alphabet in the row
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k])):
                                # print("Alpha Check",pd_csv.iat[i,k])
                                row_alphabet_check = True
                                break
                        if (row_alphabet_check):
                            #print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]

# Identifying index of Principal
i = 0
for j in range(pd_csv.shape[1]):
    if ('Principal' in str(pd_csv.iat[i, j])):
        principal_index = j



[1, 11, 153, 484, 518, 526]
Subtotal [9, 150, 479]
Type Investment 4


In [82]:
#Extracting the column data after principal
pd_csv_principal = pd_csv.iloc[:, principal_index:]

pd_csv_principal.iloc[1:,:] = pd_csv_principal.iloc[1:,:].replace(r"\(\d+\)", np.nan, regex=True)         
    
#Sweeping everyhing to left in extracted df
pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))
pd_csv_principal = pd_csv_principal.iloc[:,1:]

#Dropping columns with Nulls
pd_csv_principal = pd_csv_principal[pd_csv_principal.columns[pd_csv_principal.isnull().mean() < 0.75]]

pd_csv_principal.replace('na', np.nan, inplace=True)
#Pushing to right if only one null is present
pd_csv_principal[pd_csv_principal.isnull().sum(axis=1)==1] = pd_csv_principal[pd_csv_principal.isnull().sum(axis=1)==1].shift(1,axis=1)

pd_csv = pd_csv.iloc[:, :principal_index - 1].reset_index()
pd_csv = pd_csv.iloc[:,1:]

# perc = 100  # Like N %
# min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# # Dropping columns which have 100% NaN
# pd_csv = pd_csv.dropna(axis=1,
#                        thresh=min_count)
#Concat everything together
pd_csv = pd.concat([pd_csv, pd_csv_principal], axis=1)

new_header = pd_csv.iloc[0]  # grab the first row for the header
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
pd_csv = pd_csv.fillna("")
try:
    for col in pd_csv.columns:
        #print("Column", col)
        pd_csv[col] = pd_csv[col].astype(str).str.replace(r"\(\d+\)", "", regex=True)
except:
    print("Bracket digits not removed")

C:\Users\jaiso\AppData\Local\Temp\ipykernel_24676\874757368.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [84]:
pd_csv

,Levels,Portfolio Company,Business Description,Type of Investment,Investment Date,Shares/Units,Rate,Principal,Cost,Fair Value
1,Control Investments,Copper Trail Fund Investments,Investment Partnership,,,,,,,
2,Control Investments,Copper Trail Fund Investments,Investment Partnership,"LP Interests (CTMH, LP)",7/17/2017,38.8%,,,835,710
3,Control Investments,GRT Rubber Technologies LLC,Manufacturer of Engineered Rubber Products,,,,,,,
4,Control Investments,GRT Rubber Technologies LLC,Manufacturer of Engineered Rubber Products,Secured Debt,12/19/2014,,9.06% (L+8.00%),19152,18953,19152
5,Control Investments,GRT Rubber Technologies LLC,Manufacturer of Engineered Rubber Products,Member Units,12/19/2014,2896,,,6435,22750
...,...,...,...,...,...,...,...,...,...,...
472,Non-Control/Non-Affiliate Investments,"YS Garments, LLC",Designer and Provider of,Secured Debt,8/22/2018,,"6.54% (L+5.50%, Floor 1.00%)",6376,6351,6153
473,Non-Control/Non-Affiliate Investments,"Zips Car Wash, LLC",Express Car Wash Operator,,,,,,,
474,Non-Control/Non-Affiliate Investments,"Zips Car Wash, LLC",Express Car Wash Operator,Secured Debt,2/11/2022,,"8.26%(L+7.25%, Floor 1.00%)",2400,2355,2355
475,Non-Control/Non-Affiliate Investments,"Zips Car Wash, LLC",Express Car Wash Operator,Secured Debt,2/11/2022,,8.72%,548,543,543


In [89]:
(pd_csv.shape[1])

10


C:\Users\jaiso\AppData\Local\Temp\ipykernel_24676\1871631708.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header
